In [6]:
import pyomo.environ as pyo

## Original Problem

In [7]:
# Problem 4
def build_stochastic_farmers():
    m = pyo.ConcreteModel()
    I = [1, 2, 3]  # Scenarios

    # Decision variables
    m.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for wheat
    m.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for corn
    m.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of land for sugar beets

    m.y1 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of wheat purchased
    m.y2 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of corn purchased

    m.w1 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of wheat sold
    m.w2 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of corn sold
    m.w3 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a favorable price
    m.w4 = pyo.Var(I, within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a lower price

    # Objective function
    def obj_rule(m):
        # Ensure that coefficients correctly reflect costs and revenues for each scenario
        return sum([150*m.x1, 230*m.x2, 260*m.x3]) + sum([1/3 * (238*m.y1[i] - 170*m.w1[i] + 210*m.y2[i] - 150*m.w2[i] - 36*m.w3[i] - 10*m.w4[i]) for i in I])
    
    m.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Constraints
    m.cons1 = pyo.Constraint(expr=m.x1 + m.x2 + m.x3 <= 500)  # Total land constraint

    # Wheat production/purchase constraints for each scenario
    m.cons21 = pyo.Constraint(expr=3 * m.x1 + m.y1[1] - m.w1[1] >= 200)
    m.cons2 = pyo.Constraint(expr=2.5 * m.x1 + m.y1[2] - m.w1[2] >= 200)
    m.cons22 = pyo.Constraint(expr=2 * m.x1 + m.y1[3] - m.w1[3] >= 200)

    # Corn production/purchase constraints for each scenario
    m.cons31 = pyo.Constraint(expr=3.6 * m.x2 + m.y2[1] - m.w2[1] >= 240)
    m.cons3 = pyo.Constraint(expr=3 * m.x2 + m.y2[2] - m.w2[2] >= 240)
    m.cons32 = pyo.Constraint(expr=2.4 * m.x2 + m.y2[3] - m.w2[3] >= 240)

    # Sugar beets sale constraints for each scenario
    m.cons41 = pyo.Constraint(expr=m.w3[1] + m.w4[1] <= 24 * m.x3)
    m.cons4 = pyo.Constraint(expr=m.w3[2] + m.w4[2] <= 20 * m.x3)
    m.cons42 = pyo.Constraint(expr=m.w3[3] + m.w4[3] <= 16 * m.x3)

    m.cons5 = pyo.Constraint(I, rule=lambda m, i: m.w3[i] <= 6000)  # Sugar beets favorable price sale limit

    return m

m = build_stochastic_farmers()
solver = pyo.SolverFactory('gurobi')
solver.solve(m)

# Print results
print("Acres of land devoted to wheat: ", pyo.value(m.x1))
print("Acres of land devoted to corn: ", pyo.value(m.x2))
print("Acres of land devoted to sugar beets: ", pyo.value(m.x3))
print("Tons of wheat to purchased: ", [pyo.value(m.y1[i]) for i in m.y1])
print("Tons of corn to purchased: ", [pyo.value(m.y2[i]) for i in m.y2])
print("Tons of wheat to sold: ", [pyo.value(m.w1[i]) for i in m.w1])
print("Tons of corn to sold: ", [pyo.value(m.w2[i]) for i in m.w2])
print("Tons of sugar beets to sold at a favorable price: ", [pyo.value(m.w3[i]) for i in m.w3])
print("Tons of sugar beets to sold at a lower price: ", [pyo.value(m.w4[i]) for i in m.w4])
print("Profit: ", -pyo.value(m.obj))

Acres of land devoted to wheat:  170.0
Acres of land devoted to corn:  80.0
Acres of land devoted to sugar beets:  250.0
Tons of wheat to purchased:  [0.0, 0.0, 0.0]
Tons of corn to purchased:  [0.0, 0.0, 48.00000000000003]
Tons of wheat to sold:  [310.0, 225.0, 140.0]
Tons of corn to sold:  [48.0, 0.0, 0.0]
Tons of sugar beets to sold at a favorable price:  [6000.0, 5000.0, 4000.0]
Tons of sugar beets to sold at a lower price:  [0.0, 0.0, 0.0]
Profit:  108390.0


## Generalized Benders Decomposition Application

In [41]:
# Problem 5
def build_master_problem():
    master = pyo.ConcreteModel()

    # Decision variables for acres of land
    master.x1 = pyo.Var(within=pyo.NonNegativeReals)
    master.x2 = pyo.Var(within=pyo.NonNegativeReals)
    master.x3 = pyo.Var(within=pyo.NonNegativeReals)

    # Expected cost from subproblems
    master.theta = pyo.Var(within=pyo.Reals, initialize=0)

    # Total land constraint
    master.total_land = pyo.Constraint(expr=master.x1 + master.x2 + master.x3 <= 500)

    # Objective: Minimize the cost of land allocation plus the expected cost from subproblems
    master.obj = pyo.Objective(expr=150*master.x1 + 230*master.x2 + 260*master.x3 + master.theta, sense=pyo.minimize)

    master.benders_cuts = pyo.ConstraintList()
    
    return master

master = build_master_problem()

In [42]:
def build_subproblem(fixed_x1, fixed_x2, fixed_x3, scenario_index):
    sub = pyo.ConcreteModel()

    # Indices for the scenarios
    I = [1, 2, 3]

    # Decision variables for purchasing and selling
    sub.y1 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat purchased
    sub.y2 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn purchased
    sub.w1 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat sold
    sub.w2 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn sold
    sub.w3 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a favorable price
    sub.w4 = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold at a lower price

    # Constraints adapted from the original problem, now using fixed x values
    sub.cons_wheat = pyo.Constraint(expr=3 * fixed_x1 + sub.y1 - sub.w1 >= 200)
    sub.cons_corn = pyo.Constraint(expr=3 * fixed_x2 + sub.y2 - sub.w2 >= 240)
    sub.cons_beets_favorable = pyo.Constraint(expr=sub.w3 <= 6000)
    sub.cons_beets_total = pyo.Constraint(expr=sub.w3 + sub.w4 <= 20 * fixed_x3)

    # Objective function for the subproblem, scenario-specific
    sub.obj = pyo.Objective(expr=238*sub.y1 - 170*sub.w1 + 210*sub.y2 - 150*sub.w2 - 36*sub.w3 - 10*sub.w4, sense=pyo.minimize)

    sub.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    return sub

# Example fixed values from a solved master problem or a specific scenario analysis
fixed_x1 = 120
fixed_x2 = 80
fixed_x3 = 300

# Build and solve the subproblem for a specific scenario
subproblem_scenario_1 = build_subproblem(fixed_x1, fixed_x2, fixed_x3, scenario_index=1)
sub_solver = pyo.SolverFactory('gurobi')  
sub_solver.solve(subproblem_scenario_1)

# Print subproblem results for scenario 1
print("Tons of wheat to purchased (Scenario 1): ", pyo.value(subproblem_scenario_1.y1))
print("Tons of corn to purchased (Scenario 1): ", pyo.value(subproblem_scenario_1.y2))
print("Tons of wheat to sold (Scenario 1): ", pyo.value(subproblem_scenario_1.w1))
print("Tons of corn to sold (Scenario 1): ", pyo.value(subproblem_scenario_1.w2))
print("Tons of sugar beets to sold at a favorable price (Scenario 1): ", pyo.value(subproblem_scenario_1.w3))
print("Tons of sugar beets to sold at a lower price (Scenario 1): ", pyo.value(subproblem_scenario_1.w4))
print("Profit (Scenario 1): ", -pyo.value(subproblem_scenario_1.obj))
subproblem_scenario_1.dual.pprint()

Tons of wheat to purchased (Scenario 1):  0.0
Tons of corn to purchased (Scenario 1):  0.0
Tons of wheat to sold (Scenario 1):  160.0
Tons of corn to sold (Scenario 1):  0.0
Tons of sugar beets to sold at a favorable price (Scenario 1):  6000.0
Tons of sugar beets to sold at a lower price (Scenario 1):  0.0
Profit (Scenario 1):  243200.0
dual : Direction=IMPORT, Datatype=FLOAT
    Key                  : Value
    cons_beets_favorable : -26.0
        cons_beets_total : -10.0
               cons_corn : 210.0
              cons_wheat : 170.0


In [44]:
def add_optimality_cut(master, scenario_profit, dual_values, fixed_values):
    dual_wheat, dual_corn, dual_beets_fav, dual_beets_total = dual_values
    fixed_x1, fixed_x2, fixed_x3 = fixed_values

    # Construct the cut expression
    cut_expr = (scenario_profit - 
                dual_wheat * (3 * fixed_x1 - 200) - 
                dual_corn * (3 * fixed_x2 - 240) + 
                dual_beets_fav * 6000 - 
                dual_beets_total * (20 * fixed_x3 - 6000))

    # Add the cut to the master problem's Benders' cuts ConstraintList
    master.benders_cuts.add(expr=master.theta >= cut_expr)

dual_values = [
    subproblem_scenario_1.dual[subproblem_scenario_1.cons_wheat], 
    subproblem_scenario_1.dual[subproblem_scenario_1.cons_corn], 
    subproblem_scenario_1.dual[subproblem_scenario_1.cons_beets_favorable],
    subproblem_scenario_1.dual[subproblem_scenario_1.cons_beets_total]
]

fixed_values = [fixed_x1, fixed_x2, fixed_x3]
scenario_profit = -pyo.value(subproblem_scenario_1.obj)  # Assuming profit is the negative of the objective value

# Add the optimality cut to the master problem
add_optimality_cut(master, scenario_profit, dual_values, fixed_values)

solver.solve(master)
print("Acres of land devoted to wheat: ", pyo.value(master.x1))
print("Acres of land devoted to corn: ", pyo.value(master.x2))
print("Acres of land devoted to sugar beets: ", pyo.value(master.x3))

Acres of land devoted to wheat:  0.0
Acres of land devoted to corn:  0.0
Acres of land devoted to sugar beets:  0.0


In [45]:
subproblem_scenario_2 = build_subproblem(0, 0, 0, scenario_index=2)

sub_solver = pyo.SolverFactory('gurobi')  
sub_solver.solve(subproblem_scenario_2)

# Print subproblem results for scenario 2
print("Tons of wheat to purchased (Scenario 2): ", pyo.value(subproblem_scenario_2.y1))
print("Tons of corn to purchased (Scenario 2): ", pyo.value(subproblem_scenario_2.y2))
print("Tons of wheat to sold (Scenario 2): ", pyo.value(subproblem_scenario_2.w1))
print("Tons of corn to sold (Scenario 2): ", pyo.value(subproblem_scenario_2.w2))
print("Tons of sugar beets to sold at a favorable price (Scenario 2): ", pyo.value(subproblem_scenario_2.w3))
print("Tons of sugar beets to sold at a lower price (Scenario 2): ", pyo.value(subproblem_scenario_2.w4))
print("Profit (Scenario 2): ", -pyo.value(subproblem_scenario_2.obj))
subproblem_scenario_2.dual.pprint()

Tons of wheat to purchased (Scenario 2):  200.0
Tons of corn to purchased (Scenario 2):  240.0
Tons of wheat to sold (Scenario 2):  0.0
Tons of corn to sold (Scenario 2):  0.0
Tons of sugar beets to sold at a favorable price (Scenario 2):  0.0
Tons of sugar beets to sold at a lower price (Scenario 2):  0.0
Profit (Scenario 2):  -98000.0
dual : Direction=IMPORT, Datatype=FLOAT
    Key                  : Value
    cons_beets_favorable :   0.0
        cons_beets_total : -36.0
               cons_corn : 210.0
              cons_wheat : 238.0


In [47]:
def add_scenario_2_cut(master, scenario_profit, dual_wheat, dual_corn, dual_beets_fav, dual_beets_total, fixed_x1, fixed_x2, fixed_x3):
    cut_expr = (scenario_profit - 
                dual_wheat * (3 * master.x1 - 200) - 
                dual_corn * (3 * master.x2 - 240) - 
                dual_beets_total * (20 * master.x3 - 6000))

    master.benders_cuts.add(expr=master.theta >= cut_expr)

# Scenario 2 profit and dual values
scenario_2_profit = -98000  # Negative because it's a cost
dual_values_scenario_2 = [238, 210, 0, -36]  # [dual_wheat, dual_corn, dual_beets_favorable, dual_beets_total]

# Use the function to add the optimality cut for Scenario 2
add_scenario_2_cut(master, scenario_2_profit, *dual_values_scenario_2, fixed_x1, fixed_x2, fixed_x3)

# Solve the master problem again to find new values for x1, x2, x3
solver.solve(master)
print("Acres of land devoted to wheat: ", pyo.value(master.x1))
print("Acres of land devoted to corn: ", pyo.value(master.x2))
print("Acres of land devoted to sugar beets: ", pyo.value(master.x3))

Acres of land devoted to wheat:  0.0
Acres of land devoted to corn:  0.0
Acres of land devoted to sugar beets:  0.0
